---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    towns = pd.read_csv('university_towns.txt', header=None, sep='\n', names=['RegionName', 'State'])
    #city = pd.read_fwf('university_towns.txt', header=None)
    change = 'll'
    def remake(raw):
        brack_ind = raw['RegionName'].find('(')
        global change
        if raw['RegionName'].find('[edit]') != -1:
            change = raw['RegionName'][: raw['RegionName'].find('[edit]')]
        elif brack_ind != -1:
            raw['State'] = change
            raw['RegionName'] = raw['RegionName'][: brack_ind - 1]
        else:
            raw['State'] = change
        return raw
    towns = towns.apply(remake, axis=1).dropna()
    return towns
#get_list_of_university_towns()

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',
                        skiprows=219,
                        usecols=[4,5,6],
                        names=['Quarter', 'GDP', 'GDP chained 2009 dollars']
                       )
    for index, quart in gdp['GDP'].iteritems():
        if index < 2:
            continue
        if quart < gdp['GDP'].loc[index-1] and gdp['GDP'].loc[index-1] < gdp['GDP'].loc[index-2]:
            return gdp['Quarter'].loc[index-2]
    return gdp
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',
                        skiprows=219,
                        usecols=[4,5,6],
                        names=['Quarter', 'GDP', 'GDP chained 2009 dollars']
                       )
    start = gdp[gdp['Quarter'] == get_recession_start()].index[0]
    for index, quart in gdp[start:].GDP.iteritems():
        if quart > gdp['GDP'].loc[index-1] and gdp['GDP'].loc[index-1] > gdp['GDP'].loc[index-2]  and gdp['GDP'].loc[index-2] < gdp['GDP'].loc[index-3] and gdp['GDP'].loc[index-3] < gdp['GDP'].loc[index-4]:
            return gdp['Quarter'].loc[index]
    return gdp
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    gdp = pd.read_excel('gdplev.xls',
                        skiprows=219,
                        usecols=[4,5,6],
                        names=['Quarter', 'GDP', 'GDP chained 2009 dollars']
                       )
    start = gdp[gdp['Quarter'] == get_recession_start()].index[0]
    end = gdp[gdp['Quarter'] == get_recession_end()].index[0]
        
    return gdp.iloc[gdp[start:end].GDP.idxmin()].Quarter
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    cities = pd.read_csv('City_Zhvi_AllHomes.csv')
    cities.drop(cities.columns[6:51], axis=1, inplace=True)
    cities.set_index(["State","RegionName"], inplace=True)
    def quarting(column_name):
        if column_name[-2:] in ('01','02','03'):
            new_column = column_name[:4] + 'q1'
        elif column_name[-2:] in ('04','05','06'):
            new_column = column_name[:4] + 'q2'
        elif column_name[-2:] in ('07','08','09'):
            new_column = column_name[:4] + 'q3'
        else:
            new_column = column_name[:4] + 'q4'
        return new_column
            
    cities = cities[cities.columns[5:]].groupby(quarting, axis=1).mean()
    return cities.rename(states).sort_index()
#convert_housing_data_to_quarters()

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    gdp = pd.read_excel('gdplev.xls',
                        skiprows=219,
                        usecols=[4,5,6],
                        names=['Quarter', 'GDP', 'GDP chained 2009 dollars']
                       )
    univ_towns = get_list_of_university_towns()
    cities = convert_housing_data_to_quarters()
    rec_bottom = get_recession_bottom()
    rec_start = get_recession_start()
    ind_start = list(cities.columns).index(rec_start)
    ind_end = list(cities.columns).index(rec_bottom)
    cities = cities[[ind_start, ind_end]]
    univ_towns.set_index(['State', 'RegionName'], inplace = True)
    univ_towns = cities.merge(univ_towns,  how='inner', left_index=True, right_index=True)
    univ_towns['univ_t'] = True
    cities = cities.merge(univ_towns,
                          how='outer',
                          left_index=True, right_index=True,
                          on=[rec_start, rec_bottom])
    cities['univ_t'].fillna(False, inplace = True)
    cities['price_ratio'] = cities[rec_start] / cities[rec_bottom]
    univ_towns = cities.where(cities['univ_t']==True).dropna()
    non_univ_towns = cities.where(cities['univ_t']==False).dropna()
    t, p = ttest_ind(univ_towns['price_ratio'],
                     non_univ_towns['price_ratio'])
    different = True if p < 0.01 else False
    better =  'university town' if (univ_towns['price_ratio']).mean() < (non_univ_towns['price_ratio']).mean() else 'non-university town'
    
    return (different, p, better)
run_ttest()

(True, 0.0052294069644427911, 'university town')